In [15]:
import torch.optim as optim
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder

from utils import get_splits, preprocess_splits, get_data_loaders, create_model, train_model, evaluate_model
import fma_utils

In [2]:
# import tracks, features, and genres used for training
METADATA_DIR = './data/fma_metadata'

tracks = fma_utils.load(f'{METADATA_DIR}/tracks.csv')
genres = fma_utils.load(f'{METADATA_DIR}/genres.csv')
features = fma_utils.load(f'{METADATA_DIR}/features.csv')

In [24]:
%load_ext autoreload
%autoreload 2
some_features = ['mfcc']
many_features = ['mfcc', 'chroma_stft', 'chroma_cqt', 'spectral_contrast', 'spectral_rolloff']
all_features = ['chroma_stft' , 'chroma_cqt', 'chroma_cens',
                'tonnetz', 'mfcc', 'rmse', 'zcr',
                'spectral_centroid', 'spectral_bandwidth',
                'spectral_contrast', 'spectral_rolloff']

sizes = ['small', 'medium', 'large']
feature_sets = {'some': some_features, 'many': many_features, 'all': all_features}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
def get_split_accuracy(split, batch_size=32, num_epochs=10, learning_rate=0.002):
    """Train and evaluate a model for a given train/test split."""
    # Unpack the split data
    X_train, X_test, y_train, y_test = split

    # Preprocess the splits
    label_encoder = LabelEncoder()
    X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor = preprocess_splits(X_train, X_test, y_train, y_test, label_encoder=label_encoder)

    # Get the data loaders
    train_loader, test_loader = get_data_loaders(X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, batch_size=batch_size)
    
    # Get the input size and number of classes for split
    input_size = X_train.shape[1]
    num_classes = y_train.unique().size
    
    # Create the model
    model = create_model(input_size=input_size, hidden_size=64, num_classes=num_classes)
    
    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    train_model(model, train_loader, criterion, optimizer, num_epochs=num_epochs)

    # Evaluate the model
    accuracy, _ = evaluate_model(model, test_loader, label_encoder=label_encoder)

    return accuracy 

def get_accuracies_for_size(size, feature_sets, batch_size=32, num_epochs=10, learning_rate=0.002):
    """Get the accuracies for a given size and feature set."""
    results = {}
    for feature_set_name, feature_set in feature_sets.items():
        # Get the split
        split = get_splits(size, tracks, features, feature_set)
        
        print(f'Getting accuracy for {size} split with {feature_set_name} features')
        
        # Get the accuracy for the split with specified hyperparameters
        accuracy = get_split_accuracy(split, batch_size=batch_size, num_epochs=num_epochs, learning_rate=learning_rate)
        
        # Store the results in the dictionary
        results[(size, feature_set_name)] = accuracy
    # Return the results for each feature set
    return results

### Small Dataset Models
8,000 tracks, 8 balanced genres.

In [49]:
# Get the accuracies for the small dataset
small_accuracies = get_accuracies_for_size('small', feature_sets, batch_size=32, num_epochs=30, learning_rate=0.0001)
for (size, feature_set), accuracy in small_accuracies.items():
    print(f'Accuracy for {size} split with {feature_set} features: {accuracy}')

Getting accuracy for small split with some features
Epoch 1/60, Loss: 1.8490636348724365
Epoch 2/60, Loss: 1.6980781555175781
Epoch 3/60, Loss: 1.8371131420135498
Epoch 4/60, Loss: 1.519504189491272
Epoch 5/60, Loss: 1.5869501829147339
Epoch 6/60, Loss: 1.5220348834991455
Epoch 7/60, Loss: 1.2725785970687866
Epoch 8/60, Loss: 1.367207646369934
Epoch 9/60, Loss: 1.504025936126709
Epoch 10/60, Loss: 1.3748998641967773
Epoch 11/60, Loss: 1.4764084815979004
Epoch 12/60, Loss: 1.2784087657928467
Epoch 13/60, Loss: 1.3675354719161987
Epoch 14/60, Loss: 1.4645109176635742
Epoch 15/60, Loss: 1.1107122898101807
Epoch 16/60, Loss: 1.2871872186660767
Epoch 17/60, Loss: 1.222527265548706
Epoch 18/60, Loss: 1.337254524230957
Epoch 19/60, Loss: 0.9261729121208191
Epoch 20/60, Loss: 1.4998341798782349
Epoch 21/60, Loss: 1.1933252811431885
Epoch 22/60, Loss: 1.664971113204956
Epoch 23/60, Loss: 1.2599393129348755
Epoch 24/60, Loss: 1.3964581489562988
Epoch 25/60, Loss: 1.221601128578186
Epoch 26/60, L

### Medium Dataset Models
25,000 tracks, 161 unbalanced genres.

In [62]:
# Get the accuracies for the medium dataset
medium_accuracies = get_accuracies_for_size('medium', feature_sets, batch_size=32, num_epochs=60, learning_rate=0.0001)
for (size, feature_set), accuracy in medium_accuracies.items():
    print(f'Accuracy for {size} split with {feature_set} features: {accuracy}')

Getting accuracy for medium split with some features
Epoch 1/60, Loss: 1.4634603261947632
Epoch 2/60, Loss: 1.6056618690490723
Epoch 3/60, Loss: 1.1599535942077637
Epoch 4/60, Loss: 1.3437392711639404
Epoch 5/60, Loss: 1.5640205144882202
Epoch 6/60, Loss: 1.073938250541687
Epoch 7/60, Loss: 1.30089271068573
Epoch 8/60, Loss: 1.356502652168274
Epoch 9/60, Loss: 1.0996322631835938
Epoch 10/60, Loss: 0.6991980075836182
Epoch 11/60, Loss: 1.589154839515686
Epoch 12/60, Loss: 0.8312232494354248
Epoch 13/60, Loss: 1.118707299232483
Epoch 14/60, Loss: 0.7572696208953857
Epoch 15/60, Loss: 1.1703494787216187
Epoch 16/60, Loss: 1.329420566558838
Epoch 17/60, Loss: 1.1157855987548828
Epoch 18/60, Loss: 0.7107431888580322
Epoch 19/60, Loss: 0.8692657947540283
Epoch 20/60, Loss: 1.0847314596176147
Epoch 21/60, Loss: 1.376557469367981
Epoch 22/60, Loss: 0.9327031373977661
Epoch 23/60, Loss: 1.385693907737732
Epoch 24/60, Loss: 1.0279741287231445
Epoch 25/60, Loss: 0.8566861152648926
Epoch 26/60, Lo

### Large Dataset
106,574 tracks, 161 unbalanced genres.

In [81]:
# Get the accuracies for the large dataset
large_accuracies = get_accuracies_for_size('large', feature_sets, batch_size=128, num_epochs=30, learning_rate=0.0001)
for (size, feature_set), accuracy in large_accuracies.items():
    print(f'Accuracy for {size} split with {feature_set} features: {accuracy}')

Getting accuracy for large split with some features
Epoch 1/30, Loss: 1.0564576387405396
Epoch 2/30, Loss: 0.5493659973144531
Epoch 3/30, Loss: 2.320849657058716
Epoch 4/30, Loss: 2.0172715187072754
Epoch 5/30, Loss: 0.7259894609451294
Epoch 6/30, Loss: 0.5015782117843628
Epoch 7/30, Loss: 0.3615148961544037
Epoch 8/30, Loss: 0.8284004926681519
Epoch 9/30, Loss: 1.0224089622497559
Epoch 10/30, Loss: 3.255936622619629
Epoch 11/30, Loss: 0.4932905435562134
Epoch 12/30, Loss: 3.77467679977417
Epoch 13/30, Loss: 0.7524840831756592
Epoch 14/30, Loss: 0.39015287160873413
Epoch 15/30, Loss: 0.36393171548843384
Epoch 16/30, Loss: 0.5206798315048218
Epoch 17/30, Loss: 1.0688927173614502
Epoch 18/30, Loss: 0.9408798217773438
Epoch 19/30, Loss: 0.3415001630783081
Epoch 20/30, Loss: 0.4868425726890564
Epoch 21/30, Loss: 0.8057249784469604
Epoch 22/30, Loss: 0.6484475135803223
Epoch 23/30, Loss: 0.5658823251724243
Epoch 24/30, Loss: 0.34798407554626465
Epoch 25/30, Loss: 0.35003969073295593
Epoch 2

## Results Summary

### Small Dataset
*8,000 tracks, 8 balanced genres.*

Accuracy for **small** split with **some** features: $\approx$ 0.45

Accuracy for **small** split with **many** features: $\approx$ 0.47

Accuracy for **small** split with **all** features: $\approx$ 0.47
### Medium Dataset
*25,000 tracks, 161 unbalanced genres.*

Accuracy for **medium** split with **some** features: $\approx$ 0.60

Accuracy for **medium** split with **many** features: $\approx$ 0.62

Accuracy for **medium** split with **all** features: $\approx$ 0.63

### Large Dataset
*106,574 tracks, 161 unbalanced genres*

Accuracy for **large** split with **some** features: $\approx$ 0.61

Accuracy for **large** split with **many** features: $\approx$ 0.60

Accuracy for **large** split with **all** features: $\approx$ 0.60

